Estudante: Barbara Sabrina Rovigo Salvador 

Disciplina: Aprendizado de Máquina II

Professor: Márcio Koch

Importação das bibliotecas:

In [143]:
from glob import glob
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import os
import numpy as np
import random



Carregando as imagens:

In [148]:
def getImageVector(path_image):
  #Verifica se é um arquivo com extensão .jpg
  if ('.jpg' in path_image):
    path_splitted = path_image.split('/')
    image_name = path_splitted[len(path_splitted)-1].replace('.jpg','').split('_')

    #Extraindo id e label do path
    id = image_name[0]
    label = image_name[1]
    #print('Id: ', id, ' Label: ',label)

    # Dados da Imagem
    img = cv2.imread(path_image, 0)
    #cv2_imshow(img) 

    width = 80
    height = 80

    # dsize
    dsize = (width, height)

    #redimensiona a imagem em 80 x 80
    img_resize = cv2.resize(img, dsize)
      
    #matriz transposta
    img_transp =img_resize.T
        
    #vetor coluna
    img_reshape = img_transp.reshape(width*height,1)
        
    img_convert = img_reshape.astype('float64')
    #print('Dados da Imagem: ', img_convert)
    return (id, label, img_convert)

path_image = os.path.join("/content/drive/MyDrive/ORL2/",'*.jpg') 
images = []
for i in glob(path_image):
  images.append(getImageVector(i))

t = np.array(images, dtype=object)

sortedArr = t[t[:,0].astype(int).argsort()]

tamanho_amostra = sortedArr.shape[0]

qtde_teste = 3
qtde_amostras = 10
cont = 0
amostra = []
total_imagens = []
teste = []
treino = []
a = []
i = 0

# Rotina que separa treino e teste

while i < tamanho_amostra: #100
  # Adiciona as imagens até o contador (cont) atingir o número de amostras desejado (qtde_amostras)
  amostra.append(sortedArr[i]) 
  total_imagens.append(sortedArr[i])
  cont += 1

  if (cont == qtde_amostras): 
    # escolha randômica de k índices (qtde_teste) em um range de k amostras (qtde_amostras)
    randomRows = random.sample(range(qtde_amostras), qtde_teste) 
    # percorre a lista de imagens da amostra, segundo os índices randômicos, e adiciona as k amostras em teste
    for j in randomRows:
      teste.append(amostra[j])
  
    # limpa a amostra para receber as próximas k imagens do total de imagens
    amostra.clear()
    # reinicia o contador
    cont = 0

  i += 1

for l in teste:
  for m in total_imagens:
    if (m[0] == l[0]):
      total_imagens.remove(m)

treino = total_imagens


# Cálculo da matriz com a imagem média

num_rows = 6400
imagem_media = []
media = 0
for i in range(num_rows):
  linha = []
  soma = 0
  for j in treino:
    soma = soma + j[2][i]
  media = soma/len(treino)
  linha.append(media)
  imagem_media.append(linha) # matriz 1 coluna x 6399 linhas


# Cálculo da matriz de diferenças
imagem_diff = []

for i in range(num_rows):
  linha = []
  for j in treino:
    diff_linha = j[2][i] - imagem_media[i]
    linha.append(diff_linha)
  imagem_diff.append(linha)  # Uma coluna para cada imagem de treino

# Cálculo da covariância

a = np.array(imagem_diff) # Transforma array para np.array
qtde_colunas_treino = len(imagem_diff[0])
matriz_cov = a.reshape(num_rows,qtde_colunas_treino).T.dot(a.reshape(num_rows,qtde_colunas_treino))

#Cálculo dos autovalores e autovetores

[retval, eigenvalues, eigenvectors] = cv2.eigen(matriz_cov, True)



# Cálculo das eigenfaces
eigenvectorsT = eigenvectors.T

for z in range(10, 21, 1):
  num_componentes = z

  eigenvectorsK = eigenvectorsT[:,0:num_componentes]
  evk = np.asarray(eigenvectorsK)
  diffs = np.asarray(imagem_diff)

  eigenfaces = diffs.reshape(num_rows,qtde_colunas_treino).dot(evk)
      # Aplicando uma normalização nos dados para melhorar a qualidade do reconhecimento de imagem
  cv2.normalize(eigenfaces, eigenfaces)

  # Imprimir Eigenfaces (Pode ser implementado)

  # Cálculo da Projeção
    # Número de Linhas será igual ao número de componentes principais
    # Número de Colunas será igual ao número de imagens

  projecao_treino = eigenfaces.T.dot(diffs.reshape(num_rows,qtde_colunas_treino))

  labels = np.array(treino).T[1] # Armazena os labels de treino para conferência

  # Predição
    # Diferença da imagem teste para a imagem média
  diff_teste = []
  for i in range(num_rows):
    linha = []
    for j in teste:
      diff_linha = j[2][i] - imagem_media[i]
      linha.append(diff_linha)
    diff_teste.append(linha)  # Uma coluna para cada imagem de treino
    
    # Projetando a imagem desconhecida (de teste) no mesmo espaço vetorial de treino
  qtde_colunas_teste = len(diff_teste[0])
  diff_teste_np = np.asarray(diff_teste)
  projecao_teste = eigenfaces.T.dot(diff_teste_np.reshape(num_rows,qtde_colunas_teste))

  # Calculo da Distância Euclidiana da imagem desconhecida para a imagem de treino mais próxima
  distancias = np.zeros((projecao_teste.shape[1],projecao_treino.shape[1]))
  for p_coluna in range(projecao_teste.shape[1]):
    for q_coluna in range(projecao_treino.shape[1]):
      distancias[p_coluna,q_coluna] = cv2.norm(projecao_teste[:,p_coluna], projecao_treino[:,q_coluna]) # usa por padrão distância NORM_L2 - Distancia Euclidiana

  indice_resultados = np.zeros((projecao_teste.shape[1],7), dtype=int)
  # Cria matriz com os índices encontrados para a imagem de treino e teste
  indice_resultados[:,0] = range(len(distancias)) # Posição 0, índice das imagens de teste
  indice_resultados[:,3] = distancias.argmin(axis=1) # Posição 3, índice das imagens de treino

  for i in range(len(indice_resultados)):
    indice_resultados[i,1] = teste[indice_resultados[i][0]][0] # Posição 1, Id de Teste
    indice_resultados[i,2] = teste[indice_resultados[i][0]][1] # Posição 2, Labels de Teste
    indice_resultados[i,4] = treino[indice_resultados[i][3]][0] # Posição 4, Id de Treino
    indice_resultados[i,5] = treino[indice_resultados[i][3]][1] # Posição 5, Label de Treino
    indice_resultados[i,6] = indice_resultados[i,2] == indice_resultados[i,5] # Posição 6, Treino é igual a teste? 0 ou 1
  # Aqui poderia ser impresso o "indice_resultados" para ter acesso a matriz com todos os valores acima armazenados. Um extrato do reconhecimento de cada imagem
 
  print(num_componentes, 'componentes principais, acurácia: {:.0f}'.format((np.sum(indice_resultados[:,6])/indice_resultados.shape[0])*100),'%.') 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


10 componentes principais, acurácia: 91 %.
11 componentes principais, acurácia: 91 %.
12 componentes principais, acurácia: 92 %.
13 componentes principais, acurácia: 92 %.
14 componentes principais, acurácia: 92 %.
15 componentes principais, acurácia: 92 %.
16 componentes principais, acurácia: 92 %.
17 componentes principais, acurácia: 92 %.
18 componentes principais, acurácia: 92 %.
19 componentes principais, acurácia: 92 %.
20 componentes principais, acurácia: 93 %.
